In [2]:
from google.colab import drive
drive.mount('/content/drive')
%cd /content/drive/MyDrive/Github/MachineLearning/Module_3_Bank_Proj/Brandon

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
/content/drive/MyDrive/Github/MachineLearning/Module_3_Bank_Proj/Brandon


In [3]:
import pandas as pd

data = pd.read_csv("https://raw.githubusercontent.com/byui-cse/cse450-course/master/data/housing.csv")

In [4]:
#Profiling of Dataset

#! pip install https://github.com/pandas-profiling/pandas-profiling/archive/master.zip 
from pandas_profiling import ProfileReport as pr
#profile = pr(data)
#profile.to_notebook_iframe()

In [5]:
### Geo Location Pre-Processing ###

In [6]:
import altair as alt

from altair import pipe, limit_rows, to_values
t = lambda data: pipe(data, limit_rows(max_rows=20000), to_values)
alt.data_transformers.register('custom', t)
alt.data_transformers.enable('custom')

data = data[data.lat > 45]
data = data[data.long < -110]

data_high = data[data.price > 500000]
data_low = data[data.price <= 500000]

high_chart = alt.Chart(data_high).mark_circle().encode(
    alt.X("long:Q",
          scale=alt.Scale(zero = False)),
    alt.Y("lat:Q",
          scale=alt.Scale(zero = False)),
          color = "price"
)

low_chart = alt.Chart(data_low).mark_circle().encode(
    alt.X("long:Q",
          scale=alt.Scale(zero = False)),
    alt.Y("lat:Q",
          scale=alt.Scale(zero = False)),
          color = "price"
)

high_chart


Output hidden; open in https://colab.research.google.com to view.

In [7]:
low_chart

Output hidden; open in https://colab.research.google.com to view.

In [8]:
alt.Chart(data).mark_rect().encode(
    alt.X('long:Q', bin=alt.Bin(maxbins=100)),
    alt.Y('lat:Q', bin=alt.Bin(maxbins=100)),
    alt.Color('mean(price):Q', scale=alt.Scale(scheme='greenblue'))
)

Output hidden; open in https://colab.research.google.com to view.

In [9]:
alt.Chart(data).mark_rect().encode(
    alt.X('long:Q', bin=alt.Bin(maxbins=100)),
    alt.Y('lat:Q', bin=alt.Bin(maxbins=100)),
    alt.Color('count(id):Q', scale=alt.Scale(scheme='greenblue'))
)

Output hidden; open in https://colab.research.google.com to view.

In [10]:
data['lat_bin'] = 0
data['long_bin'] = 0
for lat in range(4715,4779,1):
  if lat + 1 > 4779:
    break
  else:
    data.loc[(data.lat >= (lat/100)) & (data.lat < ((lat/100) + .01)),'lat_bin'] = lat/100
  
for long in range(-12252,-12124,2):
  if long + 1 > -12124:
    break
  else:
    data.loc[(data.long >= (long/100)) & (data.long < ((long/100) + .02)),'long_bin'] = long/100
  

In [11]:
(data.groupby(["lat_bin","long_bin"], as_index=False)
  .agg({"price":"mean", "id":"count"})
  .rename(columns = {"price":"avg_neighborhood_price", "id":"no_houses_neighborhood"}))


,lat_bin,long_bin,avg_neighborhood_price,no_houses_neighborhood
0,47.15,-121.96,687000.000000,1
1,47.15,-121.66,380000.000000,1
2,47.16,-121.98,542500.000000,2
3,47.17,-122.04,360000.000000,1
4,47.17,-122.02,230000.000000,1
...,...,...,...,...
1181,47.77,-122.06,849333.333333,3
1182,47.77,-122.04,603750.000000,2
1183,47.77,-121.94,637000.000000,1
1184,47.77,-121.92,309637.500000,2


In [12]:
data["avg_neighbor_price"] = data.groupby(["lat_bin", "long_bin"])["price"].transform("mean")

In [13]:
alt.Chart(data).mark_circle().encode(
    alt.X("avg_neighbor_price:Q", scale = alt.Scale(type='log')),
    alt.Y("price:Q", scale = alt.Scale(type='log'))
).configure_mark(
    opacity=0.1,
    color='red'
)

Output hidden; open in https://colab.research.google.com to view.

In [22]:
import numpy as np
import xgboost as xgb
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, roc_auc_score, make_scorer
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import confusion_matrix
from sklearn.metrics import plot_confusion_matrix

In [15]:
data.head()

,id,date,bedrooms,bathrooms,sqft_living,sqft_lot,floors,waterfront,view,condition,grade,sqft_above,sqft_basement,yr_built,yr_renovated,zipcode,lat,long,sqft_living15,sqft_lot15,price,lat_bin,long_bin,avg_neighbor_price
0,1565930130,20141104T000000,4,3.25,3760,4675,2.0,0,0,3,8,2740,1020,2007,0,98038,47.3862,-122.048,3280,4033,429900.0,47.38,-122.06,303497.857143
1,3279000420,20150115T000000,3,1.75,1460,7800,1.0,0,0,2,7,1040,420,1979,0,98023,47.3035,-122.382,1310,7865,233000.0,47.30,-122.40,299063.953488
2,194000575,20141014T000000,4,1.00,1340,5800,1.5,0,2,3,7,1340,0,1914,0,98116,47.5658,-122.389,1900,5800,455000.0,47.56,-122.40,527782.707317
3,2115510160,20141208T000000,3,1.75,1440,8050,1.0,0,0,3,8,1440,0,1985,0,98023,47.3187,-122.390,1790,7488,258950.0,47.31,-122.40,284485.242424
4,7522500005,20140815T000000,2,1.50,1780,4750,1.0,0,0,4,7,1080,700,1947,0,98117,47.6859,-122.395,1690,5962,555000.0,47.68,-122.40,563207.586207


Drop id, lat, long, lat_bin, long_bin, date (Have Ben add back in later once he recodes)

In [16]:
data.drop(['id','lat','long','lat_bin','long_bin','date'], axis = 1, inplace = True)

Houses that were renovated within the past 10 years recieve a 1 while houses renovated before or never recieve a 0.

In [39]:
data["renovated_10"] = np.where(data["yr_renovated"] >= 2011, 1, 0)
data.drop('yr_renovated', axis = 1, inplace = True)

KeyError: ignored

In [40]:
X = data.drop('price', axis = 1)
y = data['price']

In [41]:
X_train, X_test, y_train, y_test = train_test_split(X, y)

In [42]:
clf = xgb.XGBRegressor(objective = "reg:linear")
clf.fit(X_train,y_train)


[13:18:07] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.


XGBRegressor(base_score=0.5, booster='gbtree', colsample_bylevel=1,
             colsample_bynode=1, colsample_bytree=1, gamma=0,
             importance_type='gain', learning_rate=0.1, max_delta_step=0,
             max_depth=3, min_child_weight=1, missing=None, n_estimators=100,
             n_jobs=1, nthread=None, objective='reg:linear', random_state=0,
             reg_alpha=0, reg_lambda=1, scale_pos_weight=1, seed=None,
             silent=None, subsample=1, verbosity=1)

In [43]:
y_pred = clf.predict(X_test)
mse = mean_squared_error(y_test, y_pred)
print(np.sqrt(mse))

127601.26314117566


RMSE ~ 129,000